In [1]:
import os
import sys
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
os.environ["KERAS_BACKEND"] = "torch"
import keras
from datamodules import data_processors
sys.path.append("..")
from rusmodules import eigenvals

In [3]:
"""
Importamos el modelo 
"""
path_modelo = "models/isotropico_act_custom_3.keras"
n_pow = 20
def custom_activation(x):
    return (1/n_pow) * torch.log((1 + torch.exp(n_pow * x)) / (1 + torch.exp(n_pow * (x - 1))))
#fin función

@keras.saving.register_keras_serializable()
class CustomActivationLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CustomActivationLayer, self).__init__(**kwargs)
    
    def call(self, inputs):
        # Ensure inputs are Torch tensors
        inputs = torch.tensor(inputs) if not isinstance(inputs, torch.Tensor) else inputs
        return custom_activation(inputs)
    #fin función
#fin clase
modelo = keras.models.load_model(path_modelo,  custom_objects={"CustomActivationLayer": CustomActivationLayer})

In [4]:
N_eig = 5
features = ["eta", "beta"] + list(map(lambda x: "eig_" + str(x+1), range(N_eig)))
target = "phi_K"
raw_exp_data = pd.read_csv("../input_data/KG_Experimental_Iso.csv")
raw_exp_data.loc[4, "omega2_1"] = 2.1679666038584755

In [5]:
raw_exp_data

,K,G,rho,dx,dy,dz,omega2_0,omega2_1,omega2_2,omega2_3,omega2_4,omega2_5,omega2_6,omega2_7,omega2_8,omega2_9
0,1.663,0.7362,8.052,0.70420,0.63100,0.58393,1.485197,2.243456,2.722901,2.909486,3.162770,3.442880,3.771830,3.805168,3.946075,4.272144
1,1.315,0.5178,5.403,0.33511,0.26023,0.15492,5.171781,8.546431,13.095081,17.098761,19.098981,19.593660,19.775137,23.387424,26.975627,29.017625
2,1.000,0.6087,9.401,0.23480,0.19140,0.14840,8.507717,14.629007,17.771782,22.233325,25.365264,26.061988,27.416796,28.093850,29.148011,30.945435
3,1.146,0.6151,9.401,0.23668,0.19364,0.14553,8.507717,14.629007,17.771782,22.233325,25.365264,26.061988,27.416796,28.093850,29.148011,30.945435
4,1.317,0.3807,8.405,0.46319,0.21930,0.38864,1.320706,2.167967,3.498974,3.897721,4.139785,4.438060,4.934177,5.329428,5.784489,6.022880
5,1.691,0.7559,8.460,0.70809,0.49945,0.60214,1.383268,2.442278,2.798756,2.952491,3.243260,3.897151,3.934252,4.269236,4.353750,4.400785
6,1.691,0.7522,8.535,0.70220,0.49905,0.60370,1.371514,2.452166,2.825646,2.929712,3.187154,3.884212,0.000100,4.154756,4.274404,4.313281


In [6]:
keys_eig_exp = "omega2_"
exp_data_full = data_processors.transform_experimental_data_isotropic(raw_exp_data, N_eig, keys_eig_exp)

In [7]:
exp_data_full

,K,G,rho,dx,dy,dz,omega2_6,omega2_7,omega2_8,omega2_9,phi_K,eta,beta,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5
0,1.663,0.7362,8.052,0.70420,0.63100,0.58393,3.771830,3.805168,3.946075,4.272144,0.416762,1.769041,2.986698,2.792124,4.217628,5.118969,5.469743,5.945910,6.472508
1,1.315,0.5178,5.403,0.33511,0.26023,0.15492,19.775137,23.387424,26.975627,29.017625,0.375119,1.469757,2.147883,0.835781,1.381138,2.116219,2.763230,3.086474,3.166416
2,1.000,0.6087,9.401,0.23480,0.19140,0.14840,27.416796,28.093850,29.148011,30.945435,0.546792,1.601784,2.638089,1.581298,2.719039,3.303175,4.132425,4.714547,4.844045
3,1.146,0.6151,9.401,0.23668,0.19364,0.14553,27.416796,28.093850,29.148011,30.945435,0.492603,1.593973,2.577968,1.575172,2.708507,3.290380,4.116418,4.696285,4.825281
4,1.317,0.3807,8.405,0.46319,0.21930,0.38864,4.934177,5.329428,5.784489,6.022880,0.281396,1.533533,2.054949,0.681330,1.118418,1.805063,2.010770,2.135646,2.289521
5,1.691,0.7559,8.460,0.70809,0.49945,0.60214,3.934252,4.269236,4.353750,4.400785,0.420368,1.670323,2.769788,2.361709,4.169798,4.778427,5.040906,5.537347,6.653760
6,1.691,0.7522,8.535,0.70220,0.49905,0.60370,0.000100,4.154756,4.274404,4.313281,0.418543,1.679833,2.763130,2.354163,4.209071,4.850137,5.028764,5.470656,6.667135


In [8]:
exp_data_full["Mag"] = (exp_data_full["K"]**2 + exp_data_full["G"]**2)**0.5

In [9]:
exp_data = data_processors.preprocess_data(exp_data_full, N_eig, target, opt = False)

In [10]:
exp_data

,phi_K,eig_0,eta,beta,eig_1,eig_2,eig_3,eig_4,eig_5
0,0.265319,2.792124,1.769041,2.986698,0.662013,0.823921,0.935870,0.919917,0.918641
1,0.238808,0.835781,1.469757,2.147883,0.605139,0.652644,0.765850,0.895271,0.974753
2,0.348099,1.581298,1.601784,2.638089,0.581565,0.823159,0.799331,0.876526,0.973267
3,0.313601,1.575172,1.593973,2.577968,0.581565,0.823159,0.799331,0.876526,0.973267
4,0.179142,0.681330,1.533533,2.054949,0.609191,0.619601,0.897697,0.941527,0.932792
5,0.267614,2.361709,1.670323,2.769788,0.566384,0.872630,0.947930,0.910347,0.832213
6,0.266452,2.354163,1.679833,2.763130,0.559307,0.867825,0.964479,0.919225,0.820541


In [11]:
X_exp = exp_data[features]
y_exp = exp_data[target]
y_gorro = modelo.predict(X_exp)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 513ms/step


In [12]:
exp_data["phi_K_pred"] = y_gorro
exp_data["Mag"] = exp_data_full["Mag"]
exp_data["K"] = exp_data_full["K"]
exp_data["G"] = exp_data_full["G"]
exp_data["K_pred"] = np.ones(len(exp_data))
exp_data["G_pred"] = np.ones(len(exp_data))
exp_data["eig_0_cal"] = np.ones(len(exp_data))
shape = "Parallelepiped"
Ng = 6
for i in exp_data.index:
    x_K = np.cos((np.pi/2)*exp_data["phi_K"][i])
    x_mu = np.sin((np.pi/2)*exp_data["phi_K"][i])
    dict_k_mu = {"x_K": x_K, "x_mu": x_mu}
    eta = exp_data["eta"][i]
    beta = exp_data["beta"][i]
    relative_eigs = eigenvals.get_eigenvalues_from_crystal_structure(Ng, dict_k_mu, eta, beta, shape)
    exp_data.loc[i,"eig_0_cal"] = relative_eigs["eig"][0]
    mag_pred = (exp_data.loc[i,"eig_0"]/exp_data.loc[i,"eig_0_cal"])
    exp_data.loc[i,"K_pred"] = mag_pred*np.cos(0.5*np.pi*exp_data.loc[i,"phi_K_pred"])
    exp_data.loc[i,"G_pred"] = mag_pred*np.sin(0.5*np.pi*exp_data.loc[i,"phi_K_pred"])
#fin for 


In [13]:
exp_data

,phi_K,eig_0,eta,beta,eig_1,eig_2,eig_3,eig_4,eig_5,phi_K_pred,Mag,K,G,K_pred,G_pred,eig_0_cal
0,0.265319,2.792124,1.769041,2.986698,0.662013,0.823921,0.935870,0.919917,0.918641,0.248900,1.818670,1.663,0.7362,1.675034,0.690432,1.541121
1,0.238808,0.835781,1.469757,2.147883,0.605139,0.652644,0.765850,0.895271,0.974753,0.269382,1.413273,1.315,0.5178,1.295393,0.583380,0.588290
2,0.348099,1.581298,1.601784,2.638089,0.581565,0.823159,0.799331,0.876526,0.973267,0.253295,1.170690,1.000,0.6087,1.052896,0.442522,1.384540
3,0.313601,1.575172,1.593973,2.577968,0.581565,0.823159,0.799331,0.876526,0.973267,0.361257,1.300640,1.146,0.6151,1.100897,0.701703,1.206556
4,0.179142,0.681330,1.533533,2.054949,0.609191,0.619601,0.897697,0.941527,0.932792,0.452072,1.370920,1.317,0.3807,1.161215,0.998325,0.444917
5,0.267614,2.361709,1.670323,2.769788,0.566384,0.872630,0.947930,0.910347,0.832213,0.282158,1.852260,1.691,0.7559,1.688075,0.801348,1.263876
6,0.266452,2.354163,1.679833,2.763130,0.559307,0.867825,0.964479,0.919225,0.820541,0.239506,1.850753,1.691,0.7522,1.724398,0.681189,1.269729


In [14]:
metr_exp = data_processors.get_metrics(X_exp, y_exp, modelo)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [15]:
print(metr_exp)

{'R2': -4.0947005360125495, 'RMSE': np.float64(0.11205315678081525), 'MAE': np.float64(0.0719818094838883)}
